In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install --upgrade ultralytics --no-deps
!pip install roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="vM6qrOEJgCQAXNdbbGk0")

project = rf.workspace("objectdetection-tecsf").project("skysealand-wrqjn")
version = project.version(2)

# Correct dataset format
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 98.2 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c


Extracting Dataset Version Zip to Skysealand-2 in yolov8:: 100%|██████████| 2626/2626 [00:00<00:00, 3927.48it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from ultralytics import YOLO

model = YOLO("yolo12s")  # choose nano version

In [4]:
import os, gc, json, shutil, yaml, warnings, random, math, copy
from pathlib import Path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from ultralytics import YOLO

In [5]:
import os

print(os.listdir("/kaggle/working"))


['Skysealand-2', '__notebook__.ipynb', 'yolo12s.pt']


In [6]:
for item in os.listdir("/kaggle/working"):
    print(item)


Skysealand-2
__notebook__.ipynb
yolo12s.pt


In [7]:
import os

base = "/kaggle/working"
dataset_dir = [d for d in os.listdir(base) if d.lower().startswith("sky")][0]

print("Dataset folder:", dataset_dir)

yaml_path = f"{base}/{dataset_dir}/data.yaml"

yaml_content = f"""
train: {base}/{dataset_dir}/train/images
val: {base}/{dataset_dir}/valid/images
test: {base}/{dataset_dir}/test/images

nc: 4
names:
  - Airplane
  - Boat
  - Car
  - Ship
"""

with open(yaml_path, "w") as f:
    f.write(yaml_content)

print("Data.yaml written to:", yaml_path)


Dataset folder: Skysealand-2
Data.yaml written to: /kaggle/working/Skysealand-2/data.yaml


In [8]:
print(os.listdir(f"/kaggle/working/{dataset_dir}"))


['README.roboflow.txt', 'valid', 'README.dataset.txt', 'data.yaml', 'test', 'train']


In [9]:
for root, dirs, files in os.walk("/kaggle/working"):
    if "data.yaml" in files:
        print(os.path.join(root, "data.yaml"))


/kaggle/working/Skysealand-2/data.yaml


In [10]:
import os, shutil

base_path = "/kaggle/working/Skysealand-2"

for folder in ['train', 'valid', 'test']:
    img_dir = os.path.join(base_path, folder, 'images')
    label_dir = os.path.join(base_path, folder, 'labels')
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(label_dir, exist_ok=True)

# move misplaced images from root into train/images if any
for f in os.listdir(base_path):
    if f.endswith('.jpg') or f.endswith('.png'):
        shutil.move(os.path.join(base_path, f), os.path.join(base_path, 'train/images', f))

print("Folder structure ready for YOLO training.")

Folder structure ready for YOLO training.


In [11]:
import os
os.listdir("/kaggle/working")


['Skysealand-2', '__notebook__.ipynb', 'yolo12s.pt']

In [12]:
from ultralytics import YOLO
from pathlib import Path
import os

# Auto-detect SkySeaLand dataset folder
base = "/kaggle/working"
dataset_dir = [d for d in os.listdir(base) if "sky" in d.lower()][0]

DATA = f"{base}/{dataset_dir}/data.yaml"
OUTPUT_FOLDER = Path("/kaggle/working/yolo-output")

# Check YAML exists before training
if not Path(DATA).exists():
    raise FileNotFoundError(f"data.yaml not found at: {DATA}")

print("Using dataset:", DATA)

# Correct model name (YOLOv8)
model = YOLO("yolov8n.pt")

# Train model
model.train(
    data=DATA,
    epochs=50,
    imgsz=640,
    batch=16,
    workers=2,
    project=str(OUTPUT_FOLDER),
    name="train"
)

print("Training started successfully!")


Using dataset: /kaggle/working/Skysealand-2/data.yaml
Ultralytics 8.3.237 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/Skysealand-2/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, over

invalid value encountered in less
invalid value encountered in less


                   all        132       1992      0.848      0.762      0.819      0.511
              Airplane         37        367      0.953      0.879      0.931      0.548
                  Boat         33        657      0.805      0.738      0.793      0.491
                   Car         31        679      0.863       0.76      0.815      0.528
                  Ship         52        289      0.771      0.671      0.737       0.48
Speed: 0.1ms preprocess, 5.3ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to /kaggle/working/yolo-output/train
Training started successfully!


In [13]:
from ultralytics import YOLO
from pathlib import Path

# --- 1. Define Paths ---
# Ensure these paths match your training setup
DATA = "/kaggle/working/Skysealand-2/data.yaml"
OUTPUT_FOLDER = Path("/kaggle/working/yolo-output")
# This assumes your training run was named "train" (from project=OUTPUT_FOLDER, name="train")
# If you used a different name, change "train" here.
TRAIN_RUN_PATH = OUTPUT_FOLDER / "train" 
BEST_WEIGHTS_PATH = TRAIN_RUN_PATH / "weights" / "best.pt"

# --- 2. Load the Best Model ---
# Load the best model weights saved during training
print(f"Loading best model from: {BEST_WEIGHTS_PATH}")
model = YOLO(BEST_WEIGHTS_PATH)

# --- 3. Evaluate on the Test Set ---
print("Evaluating model on the 'test' split...")
metrics = model.val(
    data=str(DATA),
    split="test",
    plots=True,                 # Save confusion matrix, P-R curve, etc.
    project=str(OUTPUT_FOLDER),
    name="test_evaluation"      # Saves results to /kaggle/working/yolo-output/test_evaluation
)

# --- 4. Extract, Calculate, and Print Metrics ---
print("Extracting metrics...")
map50_95 = metrics.box.map    # mAP@0.5:0.95
map50 = metrics.box.map50     # mAP@0.5
precision = metrics.box.mp    # Precision
recall = metrics.box.mr       # Recall

# Calculate F1 Score
# F1 = 2 * (Precision * Recall) / (Precision + Recall)
f1 = 2 * (precision * recall) / (precision + recall + 1e-6) # Add 1e-6 to avoid division by zero

# Print the results in a clean format
print("\n--- Test Set Evaluation Results ---")
print(f"Model: {str(BEST_WEIGHTS_PATH)}")
print(f"mAP@0.5:0.95: {map50_95:.4f}")
print(f"mAP@0.5       : {map50:.4f}")
print(f"Precision     : {precision:.4f}")
print(f"Recall        : {recall:.4f}")
print(f"F1 Score      : {f1:.4f}")
print("-----------------------------------")

Loading best model from: /kaggle/working/yolo-output/train/weights/best.pt
Evaluating model on the 'test' split...
Ultralytics 8.3.237 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2242.8±828.7 MB/s, size: 250.4 KB)
val: Scanning /kaggle/working/Skysealand-2/test/labels... 127 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 127/127 1.4Kit/s 0.1s
val: New cache created: /kaggle/working/Skysealand-2/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 2.7it/s 3.0s


invalid value encountered in less
invalid value encountered in less


                   all        127       2075      0.839      0.755      0.833      0.496
              Airplane         52        553      0.944      0.917      0.949      0.534
                  Boat         23        334      0.754      0.719      0.781      0.478
                   Car         24        798      0.834      0.742      0.832      0.508
                  Ship         46        390      0.825      0.641      0.772      0.463
Speed: 1.4ms preprocess, 8.7ms inference, 0.0ms loss, 7.9ms postprocess per image
Results saved to /kaggle/working/yolo-output/test_evaluation
Extracting metrics...

--- Test Set Evaluation Results ---
Model: /kaggle/working/yolo-output/train/weights/best.pt
mAP@0.5:0.95: 0.4956
mAP@0.5       : 0.8332
Precision     : 0.8391
Recall        : 0.7545
F1 Score      : 0.7946
-----------------------------------


**Self Supervised - DINO**

In [14]:
!pip install torch torchvision scikit-learn matplotlib ultralytics umap

ERROR: Could not find a version that satisfies the requirement umap (from versions: none)
ERROR: No matching distribution found for umap


In [15]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
from pathlib import Path
import random

class DinoWeedsDataset(Dataset):
    def __init__(self, img_dirs, global_size=224, local_size=96):
        self.img_paths = []
        for d in img_dirs:
            d = Path(d)
            for ext in ("*.jpg", "*.jpeg", "*.png"):
                self.img_paths.extend(list(d.glob(ext)))

        assert len(self.img_paths) > 0, "No images found for DINO"

        # ----- Global crops -----
        self.global_transform = transforms.Compose([
            transforms.RandomResizedCrop(global_size, scale=(0.4, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
            transforms.RandomGrayscale(p=0.2),
            transforms.ToTensor(),
        ])

        # ----- Local crops -----
        self.local_transform = transforms.Compose([
            transforms.RandomResizedCrop(local_size, scale=(0.05, 0.4)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
            transforms.RandomGrayscale(p=0.2),
            transforms.ToTensor(),
        ])

        self.num_local_crops = 6

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert("RGB")

        crops = []
        crops.append(self.global_transform(img))
        crops.append(self.global_transform(img))

        for _ in range(self.num_local_crops):
            crops.append(self.local_transform(img))

        return crops


In [16]:
#============================================================
# Wrap YOLOv12n backbone as a feature encoder 
# ============================================================

from ultralytics.nn.tasks import DetectionModel

# Load a fresh YOLOv12n model as encoder base
yolo_for_ssl = YOLO("yolo12n.pt")
yolo_model: DetectionModel = yolo_for_ssl.model  # DetectionModel

class Yolo12BackboneEncoder(nn.Module):
    """
    Encoder wrapper around the YOLOv12n DetectionModel.

    - Runs the full detection model forward
    - Uses a forward hook on a high-level feature layer (model[-2])
    - Applies global average pooling to get a feature vector
    """
    def __init__(self, det_model: DetectionModel):
        super().__init__()
        self.det_model = det_model
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self._feature = None

        # Choose a high-level feature layer just before the Detect head
        # This index works well in recent Ultralytics models
        self.hook_layer = self.det_model.model[-2]
        self.hook_layer.register_forward_hook(self._hook)

    def _hook(self, module, inp, out):
        # out can be tensor or list/tuple of tensors
        if isinstance(out, (list, tuple)):
            self._feature = out[-1]
        else:
            self._feature = out

    def forward(self, x):
        # Run full YOLO model; hook captures intermediate feature map
        _ = self.det_model(x)
        feats = self._feature
        if feats is None:
            raise RuntimeError("Hook did not capture features – check hook layer index.")
        x = self.pool(feats)        # [B, C, 1, 1]
        x = torch.flatten(x, 1)     # [B, C]
        return x

encoder_dino = Yolo12BackboneEncoder(yolo_model).to(device)
encoder_byol = Yolo12BackboneEncoder(yolo_model).to(device)

print("Encoder output dim example:")
dummy = torch.randn(1, 3, IMAGE_SIZE, IMAGE_SIZE).to(device)
print(encoder_dino(dummy).shape)

NameError: name 'nn' is not defined

In [ ]:
# ============================================================
# DINOv3 SSL pretraining (tuned)
# ============================================================
import lightly_train
from pathlib import Path

# Unlabeled data: use your unlabeled weeds pool
UNLABELED_DIR = SSL_ROOT / "train_unlabeled" / "images"  # already created in SSL split
PRETRAIN_OUT_DINOV3 = WORK_DIR / "ssl_pretrain_dinov3"
PRETRAIN_OUT_DINOV3.mkdir(parents=True, exist_ok=True)

# Path to DINOv3 teacher weights dataset (you must attach this as a Kaggle dataset)
DINOV3_WEIGHTS_PATH = Path("/kaggle/input/dinov3-weights/dinov3_vitb16_pretrain.pth")
print("Unlabeled dir exists:", UNLABELED_DIR.exists())
print("Pretrain out:", PRETRAIN_OUT_DINOV3)

def run_dinov3_ssl_pretrain(
    teacher_name="dinov3/vitb16",
    teacher_path=DINOV3_WEIGHTS_PATH,
    epochs=40,           # was 2
    batch_size=32,
):
    print(f"Starting DINOv3→YOLOv12n distillation")
    print(f"Teacher: {teacher_name}")
    print(f"Teacher weights: {teacher_path}")

    lightly_train.train(
        out=str(PRETRAIN_OUT_DINOV3),
        data=str(UNLABELED_DIR),
        model="ultralytics/yolo12n.pt",
        method="distillation",
        method_args={
            "teacher": teacher_name,
            "teacher_url": str(teacher_path),
        },
        epochs=epochs,
        batch_size=batch_size,
        precision="16-mixed",
        overwrite=True,
        num_workers=0,
    )
    print("DINOv3 SSL pretraining finished.")

# run longer SSL pretraining
run_dinov3_ssl_pretrain(epochs=40, batch_size=32)
print("DINOv3 SSL pretraining outputs in:", PRETRAIN_OUT_DINOV3)

In [ ]:
import torch, gc

torch.cuda.empty_cache()
gc.collect()


In [ ]:
from ultralytics import YOLO
from pathlib import Path
import torch, gc

torch.cuda.empty_cache()
gc.collect()

WORK_DIR = Path("/kaggle/working")
DATA_YAML = WORK_DIR / "Skysealand-2" / "data.yaml"

yolo = YOLO("yolov8n.pt")

results = yolo.train(
    data=str(DATA_YAML),
    epochs=60,
    imgsz=512,      
    batch=4,      
    device=0,
    workers=2,
    amp=True,
    cache=False,
    patience=10,
)


In [ ]:
from pathlib import Path

base = Path("/kaggle/working")

print("Searching for YOLO results.csv files:\n")

for p in base.rglob("results.csv"):
    print(p)


In [ ]:
from pathlib import Path

base = Path("/kaggle/working")

print("Searching for all results.csv files...\n")

found_csvs = list(base.rglob("results.csv"))

if not found_csvs:
    raise FileNotFoundError("No results.csv found. Check if YOLO training completed successfully.")

for i, f in enumerate(found_csvs):
    print(f"{i+1}: {f}")

# Pick the first one (or adjust index)
csv_path = found_csvs[0]
print("\nUsing results.csv:", csv_path)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv(csv_path)
epochs = df["epoch"]

# ---- Loss ----
train_loss = df["train/box_loss"] + df["train/cls_loss"]
if "train/dfl_loss" in df.columns:
    train_loss += df["train/dfl_loss"]

val_loss = None
if {"val/box_loss", "val/cls_loss"}.issubset(df.columns):
    val_loss = df["val/box_loss"] + df["val/cls_loss"]
    if "val/dfl_loss" in df.columns:
        val_loss += df["val/dfl_loss"]

# ---- Metrics ----
map50 = df.get("metrics/mAP50(B)", None)
map5095 = df.get("metrics/mAP50-95(B)", None)

# ---- Plot ----
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, label="Train loss")
if val_loss is not None:
    plt.plot(epochs, val_loss, label="Val loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("YOLOv8n – Train vs Val Loss (SkySeaLand-2)")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
if map50 is not None:
    plt.plot(epochs, map50, label="mAP@0.5")
if map5095 is not None:
    plt.plot(epochs, map5095, label="mAP@0.5:0.95")
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.title("YOLOv8n – mAP over Epochs (SkySeaLand-2)")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
